In [1]:
import sys
sys.path.insert(0, '../scripts/')

from transactions import get_transactions
from utils import export_dictionary


LAND = {"name": "LAND", "txs_contract": "0xf87e31492faf9a91b02ee0deaad50d51d56d5d4d",
        "abi_contract": "0xa57e126b341b18c262ad25b86bb4f65b5e2ade45"}
ESTATE = {"name": "ESTATE", "txs_contract": "0x959e104e1a4db6317fa58f8295f586e1a978c297",
          "abi_contract": "0x1784ef41af86e97f8d28afe95b573a24aeda966e"}
LAND_MARKETPLACE = {"name": "Marketplace", "txs_contract": "0x8e5660b4ab70168b5a6feea0e0315cb49c8cd539",
                    "abi_contract": "0x19a8ed4860007a66805782ed7e0bed4e44fc6717"}

KITTY = {"name": "Kitties", "txs_contract": "0x06012c8cf97bead5deae237070f9587f8e7a266d",
                     "abi_contract": "0x06012c8cf97bead5deae237070f9587f8e7a266d"}
KITTY_MARKETPLACE = {"name": "Marketplace", "txs_contract": "0xb1690c08e213a35ed9bab7b318de14420fb57d8c",
         "abi_contract": "0xb1690c08e213a35ed9bab7b318de14420fb57d8c"}

# LAND one year block 12322265

txs = get_transactions(contracts=[KITTY_MARKETPLACE, KITTY], start_block=6605100, end_block=6618100)

filename = "cryptokitties_everyday"

export_dictionary(txs, "../data/transactions", filename)

Contract: Marketplace
	Block number: 6605100
	1458 transactions retrieved
	Block number: 6618079
	1 transactions retrieved
Contract: Kitties
	Block number: 6605100
	7826 transactions retrieved
	Block number: 6618097
	1 transactions retrieved


In [2]:
import pandas as pd
from traces import clean_df

df = pd.DataFrame(txs) if txs else pd.read_json(f"../data/transactions{filename}.json")

df = clean_df(df)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9088 entries, 0 to 9087
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   timeStamp               9088 non-null   object
 1   hash                    9088 non-null   object
 2   transactionIndex        9088 non-null   object
 3   from                    9088 non-null   object
 4   to                      9088 non-null   object
 5   functionName            9088 non-null   object
 6   inputFunctionName       9088 non-null   object
 7   Marketplace__tokenId    1389 non-null   object
 8   inputFunctionParams     9088 non-null   object
 9   Kitties__matronId       5109 non-null   object
 10  Kitties__sireId         5109 non-null   object
 11  Kitties__kittyId        1895 non-null   object
 12  Kitties__startingPrice  1895 non-null   object
 13  Kitties__endingPrice    1895 non-null   object
 14  Kitties__duration       1895 non-null   object
 15  Kitt

When we have more than one contract, it can happen that we would like to rename some attributes that represents the same data with the same name. This way, we can use the new attribute name to group events in traces. An example are `Kitties_kittyId` and `Marketplace_tokenId` which represents the same value.

In [ ]:
r_df = df.rename(columns={'Marketplace__tokenId': 'Kitties__kittyId'}) #rename marketplace_tokenId to Kitties_kittyId

r_df.info()

In [ ]:
def sjoin(x): return ';'.join(x[x.notnull()].astype(object))

r_df = r_df.groupby(level=0, axis=1).apply(lambda x: x.apply(sjoin, axis=1))

r_df.info()

In [ ]:
from traces import sort_df, generate_xes

r_df = sort_df(r_df, columns=['timeStamp', 'transactionIndex'])

generate_xes(r_df, filename, case_concept_name="Kitties__kittyId", concept_name="inputFunctionName")
